# 🔬 Data exploration and preparation
In this notebook, we'll examine the dataset and create a subset of it for further analysis. The dataset was relatively clean when downloaded, though we addressed some problematic delimiter issues for you. If you're interested in tackling these issues firsthand, the original dataset is available at the [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).

### 1. Loading the data
Load the three datasets and explore the data.


In [64]:
import pandas as pd
df_ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', encoding='latin-1')
df_books = pd.read_csv('data/BX-Books.csv', low_memory=False, sep=';', encoding='latin-1')
df_users = pd.read_csv('data/BX-Users.csv', low_memory=False, sep=';', encoding='latin-1')

### 2. Cleaning the data
Ensure that all reviews are linked to a book. Investigate whether there are any reviews that lack a corresponding book or user. Verify the accuracy of author names and identify any anomalies, such as users who have submitted an unusually high number of reviews. Describe the process you followed to clean the data.

In [65]:
#Ensure that all reviews are linked to an existing book
df_merged_book = pd.merge(df_ratings, df_books, on='ISBN', how='inner')
print('There are', len(df_ratings), 'different ratings in the database, while there are', len(df_merged_book), 'reviews of existing books.')
#So some books are not existing

#Ensure that all reviews are linked to an existing user
df_merged_user = pd.merge(df_ratings, df_users, on='User-ID', how='inner')
print('There are', len(df_ratings), 'different ratings in the database, while there are', len(df_merged_user), 'reviews of existing users.')
#So all users exists

#Find missing values
df_ratings.isna().sum() #No missing values
df_books.isna().sum()   #In total 6 missing values for the Author, Publisher and Large Image URL
df_users.isna().sum()   #In total 110762 missing values for the variable Age

#Frequency
df_ratings['User-ID'].value_counts()    #There is one user who gave over thousand ratings, namely User-ID = 11676

There are 1149779 different ratings in the database, while there are 1031175 reviews of existing books.
There are 1149779 different ratings in the database, while there are 1149779 reviews of existing users.


User-ID          0
Location         0
Age         110762
dtype: int64

### 3. Subsetting the data
The publication accompanied with this dataset [Improving Recommendation Lists Through Topic Diversification](http://www2.informatik.uni-freiburg.de/~cziegler/BX/WWW-2005-Preprint.pdf) by Cai-Nicolas Ziegler, Sean M. McNee, Joseph A. Konstan, Georg Lausen; describes the process of subsetting (condensation steps) the dataset as follows (p5): 

> Hence, we discarded all books missing taxonomic descriptions, along with all ratings referring to them. Next, we also removed book titles with fewer than 20 overall mentions. Only community members with at least five ratings each were kept. 

Investigate the significance of these parameters for the dataset as a whole. Additionally, decide whether to include implicit ratings (where Book-Rating equals 0) in your final dataset. Consider the potential consequences of this choice. Would you opt to exclude them prior to assessing other parameters, or would it be more appropriate to exclude them later?

Although the publication outlines the expected dimensions of the resulting dataset, it's acceptable if your findings vary at this stage.

In [71]:
#Drop review if rating is zero
df_ratings = df_ratings[df_ratings['Book-Rating'] != 0]

#Remove titles with less than 20 mentions
ISBN_freq = df_ratings['ISBN'].value_counts() >= 20
df_ratings = df_ratings[df_ratings['ISBN']. isin(ISBN_freq[ISBN_freq].index)]
df_ratings['ISBN'].value_counts()   #to check

#Only keep members with at least five ratings
user_freq = df_ratings['User-ID'].value_counts() >= 5
df_ratings = df_ratings[df_ratings['User-ID']. isin(user_freq[user_freq].index)]
df_ratings['User-ID'].value_counts()   #to check


11676     1029
16795      234
95359      201
104636     163
153662     139
          ... 
117963       5
117553       5
228154       5
116599       5
276681       5
Name: User-ID, Length: 4238, dtype: int64

### 4. Extra step
Examine the `BX-Books.csv` file specifically for the book Robots and _Empire by Isaac Asimov_. Identify any issues you come across. Would you address these issues?

Given that this could pose a problem for our dataset, consider how you would resolve it. You may need to revisit step 2 if you decide to undertake this additional step.

In [74]:
df_books[df_books['Book-Title'] == 'Robots and Empire']
#This book has been published by three different companies, in different years.

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
19463,0586062009,Robots and Empire,Isaac Asimov,1986,HarperCollins,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...,http://images.amazon.com/images/P/0586062009.0...
83090,0345328949,Robots and Empire,Isaac Asimov,1991,Del Rey Books,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...,http://images.amazon.com/images/P/0345328949.0...
136152,0385190921,Robots and Empire,Isaac Asimov,1985,Smithmark Pub,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...,http://images.amazon.com/images/P/0385190921.0...


### 5. Save the new dataset(s)
Save the dataset(s) in distinct named CSV-files for later usage. Move the file(s) to the data directory.


In [76]:
df_ratings.to_csv('data/BX-Book-Ratings-Subset.csv', index=False, sep=';')